# DCGAN
**Guideline**:
1. Pooling layer di replace dengan strided conv
2. Pake batchnorm
3. Menghilangkan FC hidden layer
4. Generator pake ReLU, kecuali outputnya pake hyperbolic tan
5. Discriminator pake LReLU

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [2]:
class Discriminator(nn.Module):
    def __init__(self, channel, features_d):
        super(Discriminator, self).__init__()
        self.d = nn.Sequential(
            # INITIAL LAYER
            # kernel size, stride, padding based on paper.
            # batch norm diskip

            # Input shape: n x channel x 64 x 64
            nn.Conv2d(channel, features_d, kernel_size=4, stride=2, padding=1, bias=False),  # 32x32
            nn.LeakyReLU(0.2),

            # Conv block layer
            self._block(features_d, features_d*2, kernel_size=4, stride=2, padding=1), # 16x16
            self._block(features_d*2, features_d*2*2, kernel_size=4, stride=2, padding=1), # 8x8
            self._block(features_d*2*2, features_d*2*2*2, kernel_size=4, stride=2, padding=1), # 4x4
            nn.Conv2d(features_d*2*2*2, 1, kernel_size=4, stride=1, padding=0, bias=False), # Real or fake
            nn.Sigmoid()
        )

    # conv block
    def _block(self, in_channels, out_channels,kernel_size, stride, padding):
        return nn.Sequential(
            # ga pake bias soalnya mau di batchnorm
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2)
        )
    
    def forward(self, x):
        return self.d(x)

class Generator(nn.Module):
    """
    Ngelakuin kebalikannya dari Discriminator. Alih alih pake conv, kita pake conv transpose 
    karena kita ingin mengubah gambar menjadi gambar yang lebih besar (upscaling). Alih-alih 
    gak pake batchnorm waktu diinput, sekarang nggak pake batchnorm di output.
    """
    def __init__(self, z_dim, channel, features_g):
        super(Generator, self).__init__()
        self.g = nn.Sequential(
            # INITIAL LAYER
            # kernel size, stride, padding based on paper.
            
            # Input shape: n x z_dim x 1 x 1

            self._block(z_dim, features_g*16, kernel_size=4, stride=1, padding=0), # 4x4
            self._block(features_g*16, features_g*8, kernel_size=4,stride=2, padding=1),  # 8x8
            self._block(features_g*8, features_g*4, kernel_size=4,stride=2, padding=1),  # 16x16
            self._block(features_g*4, features_g*2, kernel_size=4,stride=2, padding=1),  # 32x32
            nn.ConvTranspose2d(features_g*2, channel, kernel_size=4, stride=2, padding=1, bias=False), # 64x64
            nn.Tanh()
        )

    def _block(self, in_channels, out_channels,kernel_size, stride, padding):
        return nn.Sequential(
            # ga pake bias soalnya mau di batchnorm
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU() # Sesuai dengan guideline
        )
    
    def forward(self, x):
        return self.g(x)

# weight initialization based on DCGAN paper
def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight, 0.0, 0.02)

In [3]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
LEARNING_RATE = 2e-4
BATCH_SIZE = 128
IMAGE_SIZE = 64
CHANNEL = 3
Z_DIM = 100
EPOCH = 10
FEATURES_D = 64
FEATURES_G = 64

In [11]:
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5 for _ in range(CHANNEL)],
                         [0.5 for _ in range(CHANNEL)])
])

d = Discriminator(CHANNEL, FEATURES_D).to(DEVICE)
g = Generator(Z_DIM, CHANNEL, FEATURES_G).to(DEVICE)
initialize_weights(d)
initialize_weights(g)

dataset = datasets.ImageFolder(root='celeb_dataset', transform=transform)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
d_optim = optim.Adam(d.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999)) # betas based on DCGAN paper
g_optim = optim.Adam(g.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
criteon = nn.BCELoss()

fixed_noise = torch.randn(BATCH_SIZE, Z_DIM, 1, 1, device=DEVICE)
sum_writer_fake = SummaryWriter(f"runs/DCGAN/fake_data")
sum_writer_real = SummaryWriter(f"runs/DCGAN/real_data")
step = 0

In [14]:
for epoch in range(EPOCH):
    for idx, (real, _) in enumerate(loader):
        real = real.to(DEVICE)
        z_noise = torch.randn(BATCH_SIZE, Z_DIM, 1, 1, device=DEVICE)

        """
        Train Discriminator
        -> maximize log(D(x)) + log(1 - D(G(z)))
        """
        real_d = d(real).reshape(-1)
        loss_real_d = criteon(real_d, torch.ones_like(real_d))  # log(D(x)) + 0
        fake = g(z_noise)
        fake_d = d(fake).reshape(-1)
        loss_fake_d = criteon(fake_d, torch.zeros_like(fake_d)) # 0 + log(1 - D(G(z)))
        loss_d = loss_real_d + loss_fake_d # log(D(x)) + log(1 - D(G(z)))
        d_optim.zero_grad()
        loss_d.backward(retain_graph=True)
        d_optim.step()

        """
        Train Discriminator
        -> minimize log(1 - D(G(z))) but leads to vanishing gradient problem
        --> the workaround is to maximize log(D(G(z)))
        """
        out = d(fake).reshape(-1)
        loss_g = criteon(out, torch.ones_like(out))
        g_optim.zero_grad()
        loss_g.backward()
        g_optim.step()

        # Logging
        if idx == 0:
            

            with torch.no_grad():
                fake = g(fixed_noise)
                img_grid_fake = torchvision.utils.make_grid(
                    fake[:32], normalize=True)
                img_grid_real = torchvision.utils.make_grid(
                    real[:32], normalize=True)

                sum_writer_fake.add_image(
                    "Mnist Fake Images", img_grid_fake, global_step=step
                )
                sum_writer_real.add_image(
                    "Mnist Real Images", img_grid_real, global_step=step
                )
                step += 1        
        print(
            f"Epoch [{epoch}/{EPOCH}] Batch {idx}/{len(loader)} \
                      Loss D: {loss_d:.4f}, loss G: {loss_g:.4f}"
        )


Epoch [0/10] Batch 0/1583                       Loss D: 0.0843, loss G: 3.2678
Epoch [0/10] Batch 1/1583                       Loss D: 0.0825, loss G: 3.2927
Epoch [0/10] Batch 2/1583                       Loss D: 0.0848, loss G: 3.3240
Epoch [0/10] Batch 3/1583                       Loss D: 0.0810, loss G: 3.3533
Epoch [0/10] Batch 4/1583                       Loss D: 0.0779, loss G: 3.3829
Epoch [0/10] Batch 5/1583                       Loss D: 0.0736, loss G: 3.4174
Epoch [0/10] Batch 6/1583                       Loss D: 0.0713, loss G: 3.4447
Epoch [0/10] Batch 7/1583                       Loss D: 0.0670, loss G: 3.4712
Epoch [0/10] Batch 8/1583                       Loss D: 0.0656, loss G: 3.4949
Epoch [0/10] Batch 9/1583                       Loss D: 0.0654, loss G: 3.5181
Epoch [0/10] Batch 10/1583                       Loss D: 0.0628, loss G: 3.5387
Epoch [0/10] Batch 11/1583                       Loss D: 0.0614, loss G: 3.5514
Epoch [0/10] Batch 12/1583                       L

![wah](hasil_1_epoch.png)